In [1]:
!pip install datasets
!pip install update transformers

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import os
import numpy as np
from collections import Counter
import torch
import datasets
datasets.logging.set_verbosity_error()
from datasets import load_metric
# from google.colab import drive
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertModel


# # uncomment if CAN'T CONNECT TO GPU
# import psutil
# import platform

In [4]:
# add-in as occasionally receive an error which requires this to be added
# uncomment if the issue arises
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.6 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0


In [5]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

enforce_reproducibility()

In [6]:
# Preamble
import sys

sys.path.append('..')

# Preprocessing

In [7]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

df_train = train_set.to_pandas()
df_val = validation_set.to_pandas()

Downloading:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/copenlu--nlp_course_tydiqa-42333912ea665dd0/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Get train and validation data for each language
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

df_val_bengali = df_val[df_val['language'] == 'bengali']
df_val_arabic = df_val[df_val['language'] == 'arabic']
df_val_indonesian = df_val[df_val['language'] == 'indonesian']


# For testing
df_val_english = df_val[df_val['language'] == 'english']
df_train_english = df_train[df_train['language'] == 'english']


In [9]:
# Create a new dataframe with the combined documents and questions and add if they are answerable
df_train_bengali_merged = pd.DataFrame({
    'text':df_train_bengali["document_plaintext"],
    'question': df_train_bengali["question_text"],
    'answerable':(df_train_bengali["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })
df_train_arabic_merged = pd.DataFrame({
    'text': df_train_arabic["document_plaintext"],
    'question': df_train_arabic["question_text"],
    'answerable': (df_train_arabic["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
                                    })
df_train_indonesian_merged = pd.DataFrame({
    'text':df_train_indonesian["document_plaintext"],
    'question': df_train_indonesian["question_text"],
    'answerable':(df_train_indonesian["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })
df_train_english_merged = pd.DataFrame({
    'text':df_train_english["document_plaintext"],
    'question': df_train_english["question_text"],
    'answerable':(df_train_english["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })


## Same for validation data
df_val_bengali_merged = pd.DataFrame({
    'text':df_val_bengali["document_plaintext"],
    'question': df_val_bengali["question_text"],
    'answerable':(df_val_bengali["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })
df_val_arabic_merged = pd.DataFrame({
    'text': df_val_arabic["document_plaintext"],
    'question': df_val_arabic["question_text"],
    'answerable': (df_val_arabic["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
                                    })
df_val_indonesian_merged = pd.DataFrame({
    'text':df_val_indonesian["document_plaintext"],
    'question': df_val_indonesian["question_text"],
    'answerable':(df_val_indonesian["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })
df_val_english_merged = pd.DataFrame({
    'text':df_val_english["document_plaintext"],
    'question':  df_val_english["question_text"],
    'answerable':(df_val_english["annotations"].apply(lambda x: 0 if x['answer_start'] == [-1] else 1))
    })

# Tokenization of text

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_text(df, max_length=128):
    input_ids = []
    attention_masks = []

#     for row in df.iterrows():
    encoded_text = tokenizer(
        df["question"].tolist(),
        df["text"].tolist(),
        max_length=max_length,
        padding="max_length",
        truncation='only_second',
        return_attention_mask=True,
        return_tensors="pt"
    )
#         input_ids.append(encoded_text["input_ids"])
#         attention_masks.append(encoded_text["attention_mask"])

#     input_ids = torch.cat(input_ids, dim=0)
#     attention_masks = torch.cat(attention_masks, dim=0)

#     return input_ids, attention_masks
    return encoded_text


Defining model, training arguments and compute metrics

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()  # Use GPU for training


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [12]:
#define parameters for the model  - used for all models
training_args = TrainingArguments(output_dir="my_trainer",
                                  evaluation_strategy="steps",
                                  num_train_epochs=3.0,
                                  per_device_train_batch_size=16,
                                  eval_steps=500
                                  )

In [13]:
# define the compute_metrics function for the trainer
metric_f1 = load_metric('f1')
metric_ac = load_metric('accuracy')

def compute_metrics(eval_pred):
    outputs, labels = eval_pred
    predictions = np.argmax(outputs, axis=-1)
    f1 = metric_f1.compute(predictions=predictions, references=labels)
    ac = metric_ac.compute(predictions=predictions, references=labels)
    return f1 | ac

### English version - delete? + delete trainer objects as i dont use them?

In [14]:
# # In English - for testing

# train_tokenized_text = tokenize_text(df_train_english_merged)
# val_tokenized_text = tokenize_text(df_val_english_merged)


# train_input_ids = torch.cat([train_tokenized_text["input_ids"]], dim=0)
# train_attention_masks = torch.cat([train_tokenized_text["attention_mask"]], dim=0) 
# val_input_ids = torch.cat([val_tokenized_text["input_ids"]], dim=0)
# val_attention_masks = torch.cat([val_tokenized_text["attention_mask"]], dim=0)

# train_labels = torch.tensor(df_train_english_merged["answerable"].tolist())
# val_labels = torch.tensor(df_val_english_merged["answerable"].tolist())

In [15]:
# # In English - for testing

# batch_size = 32

# train_data = TensorDataset(train_input_ids.to('cuda'), train_attention_masks.to('cuda'), train_labels.to('cuda'))
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# val_data = TensorDataset(val_input_ids.to('cuda'), val_attention_masks.to('cuda'), val_labels.to('cuda'))
# val_sampler = SequentialSampler(val_data)
# val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)



In [16]:
# # define the trainer object - for English test dataset
# trainer_eng = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_data,
#     eval_dataset=val_data,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer
# )

In [17]:
# # for English testing version
# optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
# epochs = 4
# total_steps = len(train_dataloader) * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


In [18]:
# # English verson - for testing

# # Training loop
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0  # Initialize the total loss for the epoch

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
#         inputs = batch[:2]
#         labels = batch[2]

#         model.zero_grad()
#         outputs = model(*inputs, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()  # Accumulate the loss
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()
#         scheduler.step()

#     average_loss = total_loss / len(train_dataloader)  # Compute the average loss for the epoch

#     model.eval()
#     predictions = []
#     true_labels = []
#     for batch in tqdm(val_dataloader, desc=f"Evaluating Epoch {epoch + 1}"):
#         inputs = batch[:2]
#         labels = batch[2]
#         with torch.no_grad():
#             outputs = model(*inputs)
#         logits = outputs.logits
#         predictions.extend(logits.argmax(dim=1).tolist())
#         true_labels.extend(labels.tolist())

#     accuracy = accuracy_score(true_labels, predictions)
#     report = classification_report(true_labels, predictions, target_names=["Not Answerable", "Answerable"])
#     print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f} - Average Loss: {average_loss:.4f}")
#     print(report)



### Indonesian

In [19]:
# For Indonesian

train_tokenized_text_indonesian = tokenize_text(df_train_indonesian_merged)
val_tokenized_text_indonesian = tokenize_text(df_val_indonesian_merged)


train_input_ids_indonesian = torch.cat([train_tokenized_text_indonesian["input_ids"]], dim=0)
train_attention_masks_indonesian = torch.cat([train_tokenized_text_indonesian["attention_mask"]], dim=0) 
val_input_ids_indonesian = torch.cat([val_tokenized_text_indonesian["input_ids"]], dim=0)
val_attention_masks_indonesian = torch.cat([val_tokenized_text_indonesian["attention_mask"]], dim=0)

train_labels_indonesian = torch.tensor(df_train_indonesian_merged["answerable"].tolist())
val_labels_indonesian = torch.tensor(df_val_indonesian_merged["answerable"].tolist())


batch_size = 16

train_data_indonesian = TensorDataset(train_input_ids_indonesian.to('cuda'), train_attention_masks_indonesian.to('cuda'), train_labels_indonesian.to('cuda'))
train_sampler_indonesian = RandomSampler(train_data_indonesian)
train_dataloader_indonesian = DataLoader(train_data_indonesian, sampler=train_sampler_indonesian, batch_size=batch_size)

val_data_indonesian = TensorDataset(val_input_ids_indonesian.to('cuda'), val_attention_masks_indonesian.to('cuda'), val_labels_indonesian.to('cuda'))
val_sampler_indonesian = SequentialSampler(val_data_indonesian)
val_dataloader_indonesian = DataLoader(val_data_indonesian, sampler=val_sampler_indonesian, batch_size=batch_size)



In [20]:
# define the trainer object
trainer_indonesian = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_indonesian,
    eval_dataset=val_data_indonesian,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [21]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader_indonesian) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
len(train_dataloader_indonesian)

713

In [23]:
model = model.to("cuda")


# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0  # Initialize the total loss for the epoch

    for batch in tqdm(train_dataloader_indonesian, desc=f"Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]

        model.zero_grad()
        outputs = model(*inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()  # Accumulate the loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader_indonesian)  # Compute the average loss for the epoch

    model.eval()
    predictions = []
    true_labels = []
    for batch in tqdm(val_dataloader_indonesian, desc=f"Evaluating Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]
        with torch.no_grad():
            outputs = model(*inputs)
        logits = outputs.logits
        predictions.extend(logits.argmax(dim=1).tolist())
        true_labels.extend(labels.tolist())

    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=["Not Answerable", "Answerable"])
    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f} - Average Loss: {average_loss:.4f}")
    print(report)

Evaluating Epoch 1: 100%|██████████| 75/75 [00:04<00:00, 17.65it/s]


Epoch 1 - Accuracy: 0.8724 - Average Loss: 0.4172
                precision    recall  f1-score   support

Not Answerable       0.87      0.88      0.87       594
    Answerable       0.88      0.87      0.87       597

      accuracy                           0.87      1191
     macro avg       0.87      0.87      0.87      1191
  weighted avg       0.87      0.87      0.87      1191



Evaluating Epoch 2: 100%|██████████| 75/75 [00:04<00:00, 17.66it/s]


Epoch 2 - Accuracy: 0.8774 - Average Loss: 0.2568
                precision    recall  f1-score   support

Not Answerable       0.93      0.82      0.87       594
    Answerable       0.84      0.94      0.88       597

      accuracy                           0.88      1191
     macro avg       0.88      0.88      0.88      1191
  weighted avg       0.88      0.88      0.88      1191



Evaluating Epoch 3: 100%|██████████| 75/75 [00:04<00:00, 17.69it/s]


Epoch 3 - Accuracy: 0.8774 - Average Loss: 0.1782
                precision    recall  f1-score   support

Not Answerable       0.91      0.84      0.87       594
    Answerable       0.85      0.92      0.88       597

      accuracy                           0.88      1191
     macro avg       0.88      0.88      0.88      1191
  weighted avg       0.88      0.88      0.88      1191



Evaluating Epoch 4: 100%|██████████| 75/75 [00:04<00:00, 17.65it/s]

Epoch 4 - Accuracy: 0.8841 - Average Loss: 0.1150
                precision    recall  f1-score   support

Not Answerable       0.90      0.86      0.88       594
    Answerable       0.87      0.91      0.89       597

      accuracy                           0.88      1191
     macro avg       0.89      0.88      0.88      1191
  weighted avg       0.89      0.88      0.88      1191



In [24]:
model.save_pretrained("./indonesian_classification")

In [25]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/indonesian_classification/pytorch_model.bin')

/kaggle/working/indonesian_classification/pytorch_model.bin

### Bengali

In [26]:
# For Bengali

train_tokenized_text_bengali = tokenize_text(df_train_bengali_merged)
val_tokenized_text_bengali = tokenize_text(df_val_bengali_merged)


train_input_ids_bengali = torch.cat([train_tokenized_text_bengali["input_ids"]], dim=0)
train_attention_masks_bengali = torch.cat([train_tokenized_text_bengali["attention_mask"]], dim=0) 
val_input_ids_bengali = torch.cat([val_tokenized_text_bengali["input_ids"]], dim=0)
val_attention_masks_bengali = torch.cat([val_tokenized_text_bengali["attention_mask"]], dim=0)

train_labels_bengali = torch.tensor(df_train_bengali_merged["answerable"].tolist())
val_labels_bengali = torch.tensor(df_val_bengali_merged["answerable"].tolist())

batch_size = 16

train_data_bengali = TensorDataset(train_input_ids_bengali.to('cuda'), train_attention_masks_bengali.to('cuda'), train_labels_bengali.to('cuda'))
train_sampler_bengali = RandomSampler(train_data_bengali)
train_dataloader_bengali = DataLoader(train_data_bengali, sampler=train_sampler_bengali, batch_size=batch_size)

val_data_bengali = TensorDataset(val_input_ids_bengali.to('cuda'), val_attention_masks_bengali.to('cuda'), val_labels_bengali.to('cuda'))
val_sampler_bengali = SequentialSampler(val_data_bengali)
val_dataloader_bengali = DataLoader(val_data_bengali, sampler=val_sampler_bengali, batch_size=batch_size)


In [27]:
# define the trainer object
trainer_bengali = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_bengali,
    eval_dataset=val_data_bengali,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [28]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader_bengali) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
# Initialize a list to store losses
train_losses = []

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0  # Initialize the total loss for the epoch

    for batch in tqdm(train_dataloader_bengali, desc=f"Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]

        model.zero_grad()
        outputs = model(*inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()  # Accumulate the loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader_bengali)  # Compute the average loss for the epoch

    model.eval()
    predictions = []
    true_labels = []
    for batch in tqdm(val_dataloader_bengali, desc=f"Evaluating Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]
        with torch.no_grad():
            outputs = model(*inputs)
        logits = outputs.logits
        predictions.extend(logits.argmax(dim=1).tolist())
        true_labels.extend(labels.tolist())

    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=["Not Answerable", "Answerable"])
    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f} - Average Loss: {average_loss:.4f}")
    print(report)


Evaluating Epoch 1: 100%|██████████| 14/14 [00:00<00:00, 17.52it/s]


Epoch 1 - Accuracy: 0.8527 - Average Loss: 0.4222
                precision    recall  f1-score   support

Not Answerable       0.91      0.79      0.84       112
    Answerable       0.81      0.92      0.86       112

      accuracy                           0.85       224
     macro avg       0.86      0.85      0.85       224
  weighted avg       0.86      0.85      0.85       224



Evaluating Epoch 2: 100%|██████████| 14/14 [00:00<00:00, 17.48it/s]


Epoch 2 - Accuracy: 0.8616 - Average Loss: 0.2874
                precision    recall  f1-score   support

Not Answerable       0.92      0.79      0.85       112
    Answerable       0.82      0.93      0.87       112

      accuracy                           0.86       224
     macro avg       0.87      0.86      0.86       224
  weighted avg       0.87      0.86      0.86       224



Evaluating Epoch 3: 100%|██████████| 14/14 [00:00<00:00, 17.50it/s]


Epoch 3 - Accuracy: 0.8482 - Average Loss: 0.1830
                precision    recall  f1-score   support

Not Answerable       0.88      0.81      0.84       112
    Answerable       0.82      0.88      0.85       112

      accuracy                           0.85       224
     macro avg       0.85      0.85      0.85       224
  weighted avg       0.85      0.85      0.85       224



Evaluating Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 17.52it/s]

Epoch 4 - Accuracy: 0.8304 - Average Loss: 0.1191
                precision    recall  f1-score   support

Not Answerable       0.86      0.79      0.82       112
    Answerable       0.80      0.88      0.84       112

      accuracy                           0.83       224
     macro avg       0.83      0.83      0.83       224
  weighted avg       0.83      0.83      0.83       224



In [30]:
model.save_pretrained("./bengali_classification")

In [31]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/bengali_classification/pytorch_model.bin')

/kaggle/working/bengali_classification/pytorch_model.bin

### Arabic

In [32]:
# For Arabic

train_tokenized_text_arabic = tokenize_text(df_train_arabic_merged)
val_tokenized_text_arabic = tokenize_text(df_val_arabic_merged)


train_input_ids_arabic = torch.cat([train_tokenized_text_arabic["input_ids"]], dim=0)
train_attention_masks_arabic = torch.cat([train_tokenized_text_arabic["attention_mask"]], dim=0) 
val_input_ids_arabic = torch.cat([val_tokenized_text_arabic["input_ids"]], dim=0)
val_attention_masks_arabic = torch.cat([val_tokenized_text_arabic["attention_mask"]], dim=0)

train_labels_arabic = torch.tensor(df_train_arabic_merged["answerable"].tolist())
val_labels_arabic = torch.tensor(df_val_arabic_merged["answerable"].tolist())

batch_size = 16

train_data_arabic = TensorDataset(train_input_ids_arabic.to('cuda'), train_attention_masks_arabic.to('cuda'), train_labels_arabic.to('cuda'))
train_sampler_arabic = RandomSampler(train_data_arabic)
train_dataloader_arabic = DataLoader(train_data_arabic, sampler=train_sampler_arabic, batch_size=batch_size)

val_data_arabic = TensorDataset(val_input_ids_arabic.to('cuda'), val_attention_masks_arabic.to('cuda'), val_labels_arabic.to('cuda'))
val_sampler_arabic = SequentialSampler(val_data_arabic)
val_dataloader_arabic = DataLoader(val_data_arabic, sampler=val_sampler_arabic, batch_size=batch_size)


In [33]:
# define the trainer object
trainer_arabic = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_arabic,
    eval_dataset=val_data_arabic,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [34]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader_arabic) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [35]:
model = model.to("cuda")

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0  # Initialize the total loss for the epoch

    for batch in tqdm(train_dataloader_arabic, desc=f"Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]

        model.zero_grad()
        outputs = model(*inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()  # Accumulate the loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader_arabic)  # Compute the average loss for the epoch

    model.eval()
    predictions = []
    true_labels = []
    for batch in tqdm(val_dataloader_arabic, desc=f"Evaluating Epoch {epoch + 1}"):
        inputs = batch[:2]
        labels = batch[2]
        with torch.no_grad():
            outputs = model(*inputs)
        logits = outputs.logits
        predictions.extend(logits.argmax(dim=1).tolist())
        true_labels.extend(labels.tolist())

    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=["Not Answerable", "Answerable"])
    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f} - Average Loss: {average_loss:.4f}")
    print(report)

Evaluating Epoch 1: 100%|██████████| 119/119 [00:06<00:00, 17.56it/s]


Epoch 1 - Accuracy: 0.9253 - Average Loss: 0.2556
                precision    recall  f1-score   support

Not Answerable       0.94      0.91      0.92       951
    Answerable       0.92      0.94      0.93       951

      accuracy                           0.93      1902
     macro avg       0.93      0.93      0.93      1902
  weighted avg       0.93      0.93      0.93      1902



Evaluating Epoch 2: 100%|██████████| 119/119 [00:06<00:00, 17.56it/s]


Epoch 2 - Accuracy: 0.9285 - Average Loss: 0.1847
                precision    recall  f1-score   support

Not Answerable       0.95      0.90      0.93       951
    Answerable       0.91      0.96      0.93       951

      accuracy                           0.93      1902
     macro avg       0.93      0.93      0.93      1902
  weighted avg       0.93      0.93      0.93      1902



Evaluating Epoch 3: 100%|██████████| 119/119 [00:06<00:00, 17.68it/s]


Epoch 3 - Accuracy: 0.9290 - Average Loss: 0.1230
                precision    recall  f1-score   support

Not Answerable       0.93      0.93      0.93       951
    Answerable       0.93      0.93      0.93       951

      accuracy                           0.93      1902
     macro avg       0.93      0.93      0.93      1902
  weighted avg       0.93      0.93      0.93      1902



Evaluating Epoch 4: 100%|██████████| 119/119 [00:06<00:00, 17.64it/s]

Epoch 4 - Accuracy: 0.9317 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable       0.94      0.92      0.93       951
    Answerable       0.92      0.95      0.93       951

      accuracy                           0.93      1902
     macro avg       0.93      0.93      0.93      1902
  weighted avg       0.93      0.93      0.93      1902



In [36]:
model.save_pretrained("./arabic_classification")

In [37]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/arabic_classification/pytorch_model.bin')

/kaggle/working/arabic_classification/pytorch_model.bin

# Week 40 - visualisations for classifiction model
Some examples with English first to establish which we want to use. Thinking the final one is the most appropriate.

In [ ]:
# attempt to use bertvis - move to end of the notebook if it works
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2, output_attentions=True)
model.to(device)  # in this case to have on the device



# for answerable question
sentence_answerable = df_train_english_merged['text'][26]


inputs_answerable = tokenizer.encode_plus(sentence_answerable, return_tensors='pt')
input_ids_answerable = inputs_answerable['input_ids']
token_type_ids_answerable = inputs_answerable['token_type_ids']
# Move inputs to the same device as the model
input_ids_answerable = input_ids_answerable.to(device)
token_type_ids_answerable = token_type_ids_answerable.to(device)

# Pass the inputs through the model, ensuring they are on the same device
attention_answerable = model(input_ids_answerable, token_type_ids=token_type_ids_answerable)[-1]

# sentence_a_start = token_type_ids[0].tolist().index(1)
input_id_list_answerable = input_ids_answerable[0].tolist() # Batch index 0
tokens_answerable = tokenizer.convert_ids_to_tokens(input_id_list_answerable)

# for not answerable question
sentence_notanswerable = df_train_english_merged['text'][115992]

inputs_notanswerable = tokenizer.encode_plus(sentence_notanswerable, return_tensors='pt')
input_ids_notanswerable = inputs_notanswerable['input_ids']
token_type_ids_notanswerable = inputs_notanswerable['token_type_ids']
# Move inputs to the same device as the model
input_ids_notanswerable = input_ids_notanswerable.to(device)
token_type_ids_notanswerable = token_type_ids_notanswerable.to(device)

# Pass the inputs through the model, ensuring they are on the same device
attention_notanswerable = model(input_ids_notanswerable, token_type_ids=token_type_ids_notanswerable)[-1]

# sentence_a_start = token_type_ids[0].tolist().index(1)
input_id_list_notanswerable = input_ids_notanswerable[0].tolist() # Batch index 0
tokens_notanswerable = tokenizer.convert_ids_to_tokens(input_id_list_notanswerable)

In [ ]:
# answerable question
head_view(attention_answerable, tokens_answerable)

In [ ]:
# nonanswerable question
head_view(attention_notanswerable, tokens_notanswerable)

In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

model_type='bert'
model_version = 'bert-base-multilingual-cased'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)
show(model, model_type, tokenizer, sentence_answerable, layer=4, head=3)

In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

model_type='bert'
model_version = 'bert-base-multilingual-cased'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)
show(model, model_type, tokenizer, sentence_notanswerable, layer=4, head=3)

In [ ]:
# possible alternate way of visualizing

# Assuming you want to visualize attention for a specific instance (e.g., the first instance in the validation dataset)
instance_index = 0

# Prepare input for the selected instance
inputs = {
    'input_ids': val_input_ids_arabic[instance_index].unsqueeze(0).to('cuda'),
    'attention_mask': val_attention_masks_arabic[instance_index].unsqueeze(0).to('cuda')
}

# Pass the input through the model to get attentions
with torch.no_grad():
    outputs = model(**inputs)
attentions_arabic = outputs.attentions  # This will contain attention scores for different layers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you want to visualize attention for layer 3, head 0
layer_idx = 3
head_idx = 0

# Get the attention scores for the selected layer and head
attention_matrix = attentions_arabic[layer_idx][0][head_idx].cpu().numpy()

# Create a heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(attention_matrix, cmap='YlGnBu', xticklabels=False, yticklabels=False)
plt.title(f'Attention Map for Layer {layer_idx}, Head {head_idx}')
plt.show()


### Week 41 - Zero-shot cross-lingual evaluation on classifier

First, using Arabic as the training set and evaluating on Indonesian and Bengali.

Ran-out of GPUs on 4th epoch..

In [57]:
arabic_model = AutoModelForSequenceClassification.from_pretrained("./arabic_classification")
arabic_model.to('cuda')

arabic_model.eval()
predictions_bengali = []
true_labels_bengali = []
for batch in tqdm(val_dataloader_bengali, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = arabic_model(*inputs)
    logits = outputs.logits
    predictions_bengali.extend(logits.argmax(dim=1).tolist())
    true_labels_bengali.extend(labels.tolist())

accuracy_bengali = accuracy_score(true_labels_bengali, predictions_bengali)
report_bengali = classification_report(true_labels_bengali, predictions_bengali, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Bengali: Accuracy: {accuracy_bengali:.4f} - Average Loss: {average_loss:.4f}")
print(report_bengali)

arabic_model.eval()
predictions_indonesian = []
true_labels_indonesian = []
for batch in tqdm(val_dataloader_indonesian, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = arabic_model(*inputs)
    logits = outputs.logits
    predictions_indonesian.extend(logits.argmax(dim=1).tolist())
    true_labels_indonesian.extend(labels.tolist())

accuracy_indonesian = accuracy_score(true_labels_indonesian, predictions_indonesian)
report_indonesian = classification_report(true_labels_indonesian, predictions_indonesian, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Indonesian: Accuracy: {accuracy_indonesian:.4f} - Average Loss: {average_loss:.4f}")
print(report_indonesian)



Evaluating: 100%|██████████| 14/14 [00:00<00:00, 17.25it/s]


Evaluating on Bengali: Accuracy: 0.8571 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.88462   0.82143   0.85185       112
    Answerable    0.83333   0.89286   0.86207       112

      accuracy                        0.85714       224
     macro avg    0.85897   0.85714   0.85696       224
  weighted avg    0.85897   0.85714   0.85696       224



Evaluating: 100%|██████████| 75/75 [00:04<00:00, 17.74it/s]

Evaluating on Indonesian: Accuracy: 0.8841 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.90141   0.86195   0.88124       594
    Answerable    0.86838   0.90620   0.88689       597

      accuracy                        0.88413      1191
     macro avg    0.88489   0.88408   0.88406      1191
  weighted avg    0.88485   0.88413   0.88407      1191



Using Bengali as the training language, and evaluating on Arabic and Indonesian.

In [58]:
bengali_model = AutoModelForSequenceClassification.from_pretrained("./bengali_classification")

bengali_model.to('cuda')

bengali_model.eval()
predictions_arabic = []
true_labels_arabic = []
for batch in tqdm(val_dataloader_arabic, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = bengali_model(*inputs)
    logits = outputs.logits
    predictions_arabic.extend(logits.argmax(dim=1).tolist())
    true_labels_arabic.extend(labels.tolist())

accuracy_arabic = accuracy_score(true_labels_arabic, predictions_arabic)
report_arabic = classification_report(true_labels_arabic, predictions_arabic, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Arabic: Accuracy: {accuracy_arabic:.4f} - Average Loss: {average_loss:.4f}")
print(report_arabic)

bengali_model.eval()
predictions_indonesian = []
true_labels_indonesian = []
for batch in tqdm(val_dataloader_indonesian, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = bengali_model(*inputs)
    logits = outputs.logits
    predictions_indonesian.extend(logits.argmax(dim=1).tolist())
    true_labels_indonesian.extend(labels.tolist())

accuracy_indonesian = accuracy_score(true_labels_indonesian, predictions_indonesian)
report_indonesian = classification_report(true_labels_indonesian, predictions_indonesian, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Indonesian: Accuracy: {accuracy_indonesian:.4f} - Average Loss: {average_loss:.4f}")
print(report_indonesian)

Evaluating: 100%|██████████| 119/119 [00:06<00:00, 17.62it/s]


Evaluating on Arabic: Accuracy: 0.8943 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.91391   0.87066   0.89176       951
    Answerable    0.87651   0.91798   0.89676       951

      accuracy                        0.89432      1902
     macro avg    0.89521   0.89432   0.89426      1902
  weighted avg    0.89521   0.89432   0.89426      1902



Evaluating: 100%|██████████| 75/75 [00:04<00:00, 17.72it/s]

Evaluating on Indonesian: Accuracy: 0.8774 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.91026   0.83670   0.87193       594
    Answerable    0.84961   0.91792   0.88245       597

      accuracy                        0.87741      1191
     macro avg    0.87993   0.87731   0.87719      1191
  weighted avg    0.87986   0.87741   0.87720      1191



Using Indonesian as training dataset and Bengali and Arabic for validation.

In [59]:
indonesian_model = AutoModelForSequenceClassification.from_pretrained("./indonesian_classification")
indonesian_model.to('cuda')

indonesian_model.eval()
predictions_arabic = []
true_labels_arabic = []
for batch in tqdm(val_dataloader_arabic, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = indonesian_model(*inputs)
    logits = outputs.logits
    predictions_arabic.extend(logits.argmax(dim=1).tolist())
    true_labels_arabic.extend(labels.tolist())

accuracy_arabic = accuracy_score(true_labels_arabic, predictions_arabic)
report_arabic = classification_report(true_labels_arabic, predictions_arabic, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Arabic: Accuracy: {accuracy_arabic:.4f} - Average Loss: {average_loss:.4f}")
print(report_arabic)

indonesian_model.eval()
predictions_bengali = []
true_labels_bengali = []
for batch in tqdm(val_dataloader_bengali, desc=f"Evaluating Epoch {epoch + 1}"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = indonesian_model(*inputs)
    logits = outputs.logits
    predictions_bengali.extend(logits.argmax(dim=1).tolist())
    true_labels_bengali.extend(labels.tolist())

accuracy_bengali = accuracy_score(true_labels_bengali, predictions_bengali)
report_bengali = classification_report(true_labels_bengali, predictions_bengali, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Bengali: Accuracy: {accuracy_bengali:.4f} - Average Loss: {average_loss:.4f}")
print(report_bengali)

Evaluating: 100%|██████████| 119/119 [00:06<00:00, 17.62it/s]


Evaluating on Arabic: Accuracy: 0.9175 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.92688   0.90641   0.91653       951
    Answerable    0.90844   0.92850   0.91836       951

      accuracy                        0.91746      1902
     macro avg    0.91766   0.91746   0.91745      1902
  weighted avg    0.91766   0.91746   0.91745      1902



Evaluating Epoch 4: 100%|██████████| 14/14 [00:00<00:00, 17.59it/s]

Evaluating on Bengali: Accuracy: 0.8348 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.82051   0.85714   0.83843       112
    Answerable    0.85047   0.81250   0.83105       112

      accuracy                        0.83482       224
     macro avg    0.83549   0.83482   0.83474       224
  weighted avg    0.83549   0.83482   0.83474       224



In [60]:
arabic_model = AutoModelForSequenceClassification.from_pretrained("./arabic_classification")
arabic_model.to('cuda')

arabic_model.eval()
predictions_arabic = []
true_labels_arabic = []
for batch in tqdm(val_dataloader_arabic, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = arabic_model(*inputs)
    logits = outputs.logits
    predictions_arabic.extend(logits.argmax(dim=1).tolist())
    true_labels_arabic.extend(labels.tolist())

accuracy_arabic = accuracy_score(true_labels_arabic, predictions_arabic)
report_arabic = classification_report(true_labels_arabic, predictions_arabic, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Arabic: Accuracy: {accuracy_arabic:.4f} - Average Loss: {average_loss:.4f}")
print(report_arabic)

Evaluating: 100%|██████████| 119/119 [00:06<00:00, 17.46it/s]

Evaluating on Arabic: Accuracy: 0.9317 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.94378   0.91798   0.93070       951
    Answerable    0.92016   0.94532   0.93257       951

      accuracy                        0.93165      1902
     macro avg    0.93197   0.93165   0.93164      1902
  weighted avg    0.93197   0.93165   0.93164      1902



In [61]:
bengali_model = AutoModelForSequenceClassification.from_pretrained("./bengali_classification")
bengali_model.to('cuda')

bengali_model.eval()
predictions_bengali = []
true_labels_bengali = []
for batch in tqdm(val_dataloader_bengali, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = bengali_model(*inputs)
    logits = outputs.logits
    predictions_bengali.extend(logits.argmax(dim=1).tolist())
    true_labels_bengali.extend(labels.tolist())

accuracy_bengali = accuracy_score(true_labels_bengali, predictions_bengali)
report_bengali = classification_report(true_labels_bengali, predictions_bengali, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Bengali: Accuracy: {accuracy_bengali:.4f} - Average Loss: {average_loss:.4f}")
print(report_bengali)

Evaluating: 100%|██████████| 14/14 [00:00<00:00, 17.09it/s]

Evaluating on Bengali: Accuracy: 0.8304 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.86275   0.78571   0.82243       112
    Answerable    0.80328   0.87500   0.83761       112

      accuracy                        0.83036       224
     macro avg    0.83301   0.83036   0.83002       224
  weighted avg    0.83301   0.83036   0.83002       224



In [62]:
indonesian_model = AutoModelForSequenceClassification.from_pretrained("./indonesian_classification")
indonesian_model.to('cuda')

indonesian_model.eval()
predictions_indonesian = []
true_labels_indonesian = []
for batch in tqdm(val_dataloader_indonesian, desc=f"Evaluating"):
    inputs = batch[:2]
    labels = batch[2]
    with torch.no_grad():
        outputs = indonesian_model(*inputs)
    logits = outputs.logits
    predictions_indonesian.extend(logits.argmax(dim=1).tolist())
    true_labels_indonesian.extend(labels.tolist())

accuracy_indonesian = accuracy_score(true_labels_indonesian, predictions_indonesian)
report_indonesian = classification_report(true_labels_indonesian, predictions_indonesian, target_names=["Not Answerable", "Answerable"], digits=5)
print(f"Evaluating on Indonesian: Accuracy: {accuracy_indonesian:.4f} - Average Loss: {average_loss:.4f}")
print(report_indonesian)

Evaluating: 100%|██████████| 75/75 [00:04<00:00, 17.63it/s]

Evaluating on Indonesian: Accuracy: 0.8841 - Average Loss: 0.0764
                precision    recall  f1-score   support

Not Answerable    0.90426   0.85859   0.88083       594
    Answerable    0.86603   0.90955   0.88725       597

      accuracy                        0.88413      1191
     macro avg    0.88514   0.88407   0.88404      1191
  weighted avg    0.88509   0.88413   0.88405      1191



In [ ]:
0